###1.    Retrieve specific nutrient information (Twix caramel cookie bars) from the USDA DB

In [3]:
import urllib
import pandas as pd
import json
import ast

import Keys

try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [24]:
 #product query and db number retrieval        
query = urllib.quote_plus("Twix caramel cookie bars".lower())
url = "https://api.nal.usda.gov/ndb/search/?format=xml&q=" + query + "&max=1500&offset=0&api_key=" + Keys.usda_key
data = urllib.urlopen(url)
data = data.read()
root = ET.fromstring(data)
dbno = list(root)[0].find('ndbno').text #assume the 1st element correct. Ways to test that?
 #get product nutrient table
url = "https://api.nal.usda.gov/ndb/reports/?ndbno="+ dbno + "&type=f&format=json&api_key="+ Keys.usda_key
data = urllib.urlopen(url)
data = data.read()   
data = ast.literal_eval(data)
data = pd.DataFrame.from_dict(data["report"]["food"]["nutrients"])
data = data.loc[:,["group","name","unit","value"]]

In [38]:
def nutrientlists(query):
    '''A function that takes a food product name and brief description and returns a table with nutritional 
    information on the specific product and all the related ones'''

    #product query and db number retrieval        
    query = urllib.quote_plus(query.lower())
    url = "https://api.nal.usda.gov/ndb/search/?format=xml&q=" + query + "&max=1500&offset=0&api_key=" + Keys.usda_key
    data = urllib.urlopen(url) #
    data = data.read()
    root = ET.fromstring(data)
    dbno = list(root)[0].find('ndbno').text #assume the 1st element correct. Ways to test that?
    
def nutrientFromdbno:    
     #get product nutrient table
    url = "https://api.nal.usda.gov/ndb/reports/?ndbno="+ dbno + "&type=f&format=json&api_key="+ Keys.usda_key
    data = urllib.urlopen(url) #
    data = data.read()   
    data = ast.literal_eval(data)
    data = pd.DataFrame.from_dict(data["report"]["food"]["nutrients"])
    data = data.loc[:,["group","name","unit","value"]]
    return data

In [61]:
query = urllib.quote_plus("Twix caramel cookie bars".lower())
url = "https://api.nal.usda.gov/ndb/search/?format=xml&q=" + query + "&max=1500&offset=0&api_key=" + Keys.usda_key
data = urllib.urlopen(url)
data = data.read()
root = ET.fromstring(data)

In [62]:
for i in range(len(list(root))):
    print list(root)[i].find('name').text

Candies, MARS SNACKFOOD US, TWIX Caramel Cookie Bars
Candies, MARS SNACKFOOD US, TWIX Peanut Butter Cookie Bars
Candies, MARS SNACKFOOD US, TWIX chocolate fudge cookie bars
LITTLE DEBBIE, COOKIE BARS, SALTED CARAMEL, UPC: 024300042362
COOKIE & CARAMEL BAR, UPC: 053035359100
DETOUR, LEAN MUSCLE WHEY PROTEIN BAR, COOKIE DOUGH CARAMEL CRISP, UPC: 733913008756
POWERBAR, 12G PROTEIN RECOVERY BAR, COOKIES & CREAM CARAMEL CRISP, UPC: 097421620009


In [56]:
len(list(root))

299